In [1]:
###Leo & Ishan
###Date: 5th November 2024
print("Hello World!")

Hello World!


In [2]:
import chipwhisperer as cw
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np

# for graphing
from bokeh.plotting import figure, show, output_file
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span
output_file("temp_trace")

import os

In [19]:
scope = cw.scope()

In [20]:
scope.default_setup()

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.clock.clkgen_freq                  changed from 0                         to 7370129.87012987         
scope.clock.adc_freq                     changed from 0                         to 29480519.48051948        
scope.clock.extclk_monitor_enabled       changed from True                      to False                    
scope.clock.extclk_tolerance             changed from 1144409.1796875           to 13096723.705530167       
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2      

In [23]:
# scope.adc.decimate = 5
scope.adc.stream_mode = True
scope.adc.timeout = 20
# scope.adc.samples = 131070 # is this the max for buffer, anything higher needs streaming
scope.adc.samples = 8_000_000 
scope.adc.bits_per_sample = 8
scope.clock.adc_mul = 1
scope.clock.clkgen_freq = 8_000_000
scope.gain.gain = 0

In [24]:
print("sample rate is:", scope.clock.adc_mul * scope.clock.clkgen_freq / 1_000_000, "MHz")
print("sampling time is:", scope.adc.samples / (scope.clock.adc_mul * scope.clock.clkgen_freq), "seconds")

sample rate is: 8.0 MHz
sampling time is: 1.0 seconds


In [25]:
scope.trigger.module = 'ADC'
scope.trigger.level = 0

In [26]:
# print(scope)

In [27]:
num_captures = 100
capture_dir = "/media/jobs/Elements/captures/"
capture_name = "screen_off_idle"

for i in range(0, num_captures):
    scope.arm()
    if scope.capture():
        print("err")
    outfile = capture_dir + capture_name + "_" + str(i)
    np.save(outfile, scope.get_last_trace())

In [18]:
print(scope.errors)
scope.errors.clear()
print(scope.errors)

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:206) Internal error: in fast read mode but not reading FIFO! (address=43).
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:207) This happens when attempting to access (read or write) some Husky FPGA setting after the
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:208) ADC sample FIFO read has been set up; in stream mode, this is done when scope.arm() is called.
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:209) If this happens to you as a user: access scope.* settings before calling scope.arm(), not after.
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:210) If this happens to you as a developer, you can resolve this by caching the setting you need.
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:211) The error trace below will show you what led to this:


ValueError: 

In [171]:
last_trace = scope.get_last_trace()
len(last_trace)

50000000

In [183]:
last_trace

array([-0.0078125 ,  0.00390625,  0.        , ..., -0.00390625,
       -0.015625  , -0.0234375 ])

In [173]:
# averaging
window_size = 500
num_points = len(last_trace) // window_size
print("getting", f"{num_points:,}", "number of points")
# 1 million seems to be the plotting limit

getting 100,000 number of points


In [174]:
averaged_trace = [
    np.mean(last_trace[i * window_size:(i + 1) * window_size])
    for i in range(num_points)
]
xrange_averaged = list(range(num_points))

In [ ]:
# plot figure
o = figure(width=5000)
O = o.line(xrange_averaged, averaged_trace)
show(o)

In [50]:
# Delete the traces to save memory
if 'last_trace' in locals():
    del last_trace
if 'averaged_trace' in locals():
    del averaged_trace
if 'o' in locals():
    del o

In [181]:
# plotting the beginning
o = figure(width=5000)
O = o.line(list(range(0, 1000000)), last_trace[0:1000000])
show(o)

In [375]:
# Downsampling
o = figure(width=15000)

n = 100
downsampled = last_trace[::n]  # Take every nth point

xrange = list(range(len(downsampled)))
O = o.line(xrange, downsampled)
show(o)

In [195]:
scope.dis()

True